In [18]:
import os


In [19]:
%pwd

'/Users/sachinboora/Desktop/chicken-disease-classification-project'

In [20]:
os.chdir("/Users/sachinboora/Desktop/chicken-disease-classification-project")

In [21]:
%pwd

'/Users/sachinboora/Desktop/chicken-disease-classification-project'

In [22]:
import tensorflow as tf

In [23]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    training_data: Path
    params_batch_size: int
    params_image_size: list

In [25]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
import tensorflow as tf
%pwd

'/Users/sachinboora/Desktop/chicken-disease-classification-project'

In [26]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                create_directories([self.config.artifacts_root])


    def get_validation_config(self) -> EvaluationConfig:
            eval_config = EvaluationConfig(
                    path_of_model = "artifacts/training/model.keras",
                    training_data = "artifacts/data_ingestion/Chicken-fecal-images",
                    all_params = self.params,
                    params_batch_size=self.params.BATCH_SIZE,
                    params_image_size=self.params.IMAGE_SIZE,
            )

            return eval_config

In [27]:
from urllib.parse import urlparse

In [28]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return  tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)



In [29]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

[2024-04-08 16:48:09,706: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-08 16:48:09,708: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-08 16:48:09,708: INFO: common: created directory at artifacts]
Found 78 images belonging to 2 classes.
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 748ms/step - accuracy: 0.7370 - loss: nan
[2024-04-08 16:48:13,749: INFO: common: json file saved at: scores.json]
